# 2. 2D XRF deconvolution of pre-processed hdf files
## Summary
This notebook takes the pre-processed .hdf files produced with the `1_reduced_reshaped_xrf_hdf` notebook and deconvolutes to element channel images using the PyMCA package.

In [1]:
import pathlib
import shutil
import numpy as np
import pandas as pd
import h5py
import cv2
import matplotlib.pyplot as plt

from PyMca5.PyMcaPhysics.xrf.XRFBatchFitOutput import OutputBuffer
from PyMca5.PyMcaPhysics.xrf.FastXRFLinearFit import FastXRFLinearFit

In [2]:
# Set data directory to work from 
base_dir = "C:/Users/MerrickS/OneDrive/Work/2_UZH/Papers/1_MEZ_XRF"
base_dir = pathlib.Path(base_dir)

# Specify the input directory where hdf files to process are located
hdf_dir = base_dir / 'data' / 'processed' / 'xrf' / '1_reduced_reshaped_hdfs'

# Specify the config directory where config files for deconvolution are located
cfg_dir = base_dir / 'data' / 'raw' / 'xrf' / 'config'

# Make output directory for reshaped hdf files if it does not exist
out_dir = base_dir / 'data' / 'processed' / 'xrf' / '2_deconvoluted_hdfs'
out_dir.mkdir(parents=True, exist_ok=True)
print('\n Deconvoluted hdf files will be output to: \n\t', base_dir, out_dir) 

# Gather filepaths for preprocessed hdfs and config files for XRF fitting
hdf_filepaths = list(hdf_dir.glob('*.h5'))

# Read in scan and scanset metadata
df_hdf_config = pd.read_csv(hdf_dir /'preprocessed_hdf_config_files.csv')


 Deconvoluted hdf files will be output to: 
	 C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs


In [3]:
# Make hdf XRF fit config dictionary 
hdf_list = df_hdf_config['hdf_file'].tolist()
config_list = df_hdf_config['config_file'].tolist()
hdf_config_dict = dict(zip(hdf_list, config_list))

In [4]:
def hdf_dataset(hdf_filepath, dataset): 
    with h5py.File(hdf_filepath, 'r') as hdf:
        node = f"{dataset}"
        try:            
            hdf[node]
            dset = hdf[dataset]
        except KeyError:
            dset = []
            print(f'Could not find hdf dataset for {hdf_filepath}')

        dset = dset[:]
    return dset

def hdf_large_dataset(hdf_filepath, dataset, dims): 
    with h5py.File(hdf_filepath, 'r') as hdf:
        node = f"{dataset}"
        try:            
            hdf[node]
            dset = hdf[dataset]
        except KeyError:
            dset = []
            print(f'Could not find hdf dataset for {hdf_filepath}')

        dset = dset[dims[0]:dims[1],:]
    return dset

def xrf_deconvolute(hdf_file, detector, refit_state=False, weight=0):
    # Check hdf file exists
    input_hdf_fpath = hdf_dir / f'{hdf_file}.h5'
    if input_hdf_fpath.exists() is False:
        print(f'{input_hdf_fpath} does not exist')
        
    # Check detector node exists
    node_exists = False
    with h5py.File(input_hdf_fpath, 'r+') as hdf:  
        node_exists = detector in hdf
        print(f'{detector} node exists: ', node_exists)
        if node_exists == True:
            det_entries = hdf[detector].shape[0]
            print(det_entries)
            if det_entries > 100:
                node_exists = True
            else:
                node_exists = False
                print(detector, 'empty, no fit performed')
    
    if node_exists == True:
        # Check cfg file exists
        config_filepath = cfg_dir / hdf_config_dict[hdf_file]
        if config_filepath.exists() is False:
            print(f'{config_filepath} does not exist')

        # Make output subdir if does not exist
        out_sub_dir = out_dir / 'per_detector_deconvolutions' / hdf_file / detector
        out_sub_dir.mkdir(parents=True, exist_ok=True)

        # Get dataset to deconvolute and reshape to 3D array (2D array of spectra) for PyMCA
        # Establish if need to chunk data
        if det_entries > 2000000:
            z_dim = len(list(set(hdf_dataset(input_hdf_fpath, 'hrz'))))
            y_dim = int(len(list(hdf_dataset(input_hdf_fpath, 'hry')))/z_dim)
            print('large dataset', 'z', z_dim, 'y', y_dim)

            # Find chunk size            
            chunks = 10
            if z_dim % chunks == 0:
                print("Chunks:", chunks)
            else:
                print("Chunk error, can't find equal division of data")

            chunk_size = int(det_entries / chunks)
            chunk_dims = [[i*chunk_size,(i+1)*chunk_size] for i in range(chunks)]
            print(chunk_dims)

        else: 
            z_dim = len(list(set(hdf_dataset(input_hdf_fpath, 'hrz'))))
            y_dim = int(len(list(hdf_dataset(input_hdf_fpath, 'hry')))/z_dim)
            print('small dataset', 'z', z_dim, 'y', y_dim)
            chunks = 0

        # Deconvolution for chunked data
        if chunks != 0:
            for chunk in range(chunks):
                out_sub_sub_dir = out_sub_dir / f'chunk_{chunk}'
                out_sub_sub_dir.mkdir(parents=True, exist_ok=True)
                print("Deconvoluting spectral chunk:", chunk_dims[chunk])                    
                spectra_stack = hdf_large_dataset(input_hdf_fpath, detector, chunk_dims[chunk])
                print("spectra shape", spectra_stack.shape)
                spectra_stack = spectra_stack.reshape(int(z_dim/chunks), y_dim, spectra_stack.shape[-1])         
                print("reshaped spectra shape", spectra_stack.shape)        

                # Pymca fit
                if node_exists == True:
                    pymca_object = FastXRFLinearFit()    

                    print('spectra config file:', config_filepath.name)

                    FastXRFLinearFit.setFitConfigurationFile(pymca_object, str(config_filepath))

                    test_out = OutputBuffer(outputDir=out_sub_sub_dir,
                                            edf=0,
                                            tif=False,
                                            overwrite=True,
                                            outputRoot=hdf_file, 
                                            fileEntry=hdf_file
                                            )

                    FastXRFLinearFit.fitMultipleSpectra(pymca_object, 
                                                        y=spectra_stack, 
                                                        weight=weight, 
                                                        refit=refit_state, 
                                                        concentrations=False,
                                                        outbuffer=test_out)

                    # Extract tifs from hdf channels
                    output_hdf_fpath = list(out_sub_sub_dir.glob('*h5'))[0]
                    with h5py.File(output_hdf_fpath, 'r+') as hdf:
                        node = f"{output_hdf_fpath.stem}/plotselect"

                        image_dir = output_hdf_fpath.parent / "IMAGES"
                        image_dir.mkdir(parents=True, exist_ok=True)
                        for img in hdf[node].keys():
                            subnode = f"{node}/{img}"
                            dset = hdf[subnode][...]
                            outpath = str(image_dir / f'{img}.tiff')
                            cv2.imwrite(outpath, dset)

                    # Carry over the fpico normalisation mask is exists
                    output_hdf_fpath = list(out_sub_sub_dir.glob('*h5'))[0]

                    with h5py.File(input_hdf_fpath, 'r') as hdf:
                        if 'fpico_mask' in hdf:
                            fpico_mask = hdf['fpico_mask'][:]
                        else:
                            fpico_mask = False

                    print('output hdf file:', output_hdf_fpath)

                    if isinstance(fpico_mask, np.ndarray):
                        with h5py.File(output_hdf_fpath, 'r+') as hdf:
                              hdf.create_dataset(name = 'fpico_mask', data = fpico_mask)
                else:
                    pass

        # Deconvolution for non-chunked data
        if chunks == 0:
            spectra_stack = hdf_dataset(input_hdf_fpath, detector)
            print("spectra shape", spectra_stack.shape)
            spectra_stack = spectra_stack.reshape(int(z_dim), y_dim, spectra_stack.shape[-1])         
            print("reshaped spectra shape", spectra_stack.shape)        

            # Pymca fit
            if node_exists == True:
                pymca_object = FastXRFLinearFit()    

                print('spectra config file:', config_filepath.name)

                FastXRFLinearFit.setFitConfigurationFile(pymca_object, str(config_filepath))

                test_out = OutputBuffer(outputDir=out_sub_dir,
                                        edf=0,
                                        tif=True,
                                        overwrite=True,
                                        outputRoot=hdf_file, 
                                        fileEntry=hdf_file
                                        )

                FastXRFLinearFit.fitMultipleSpectra(pymca_object, 
                                                    y=spectra_stack, 
                                                    weight=weight, 
                                                    refit=refit_state, 
                                                    concentrations=False,
                                                    outbuffer=test_out)

                # Carry over the fpico normalisation mask is exists
                output_hdf_fpath = list(out_sub_dir.glob('*h5'))[0]

                with h5py.File(input_hdf_fpath, 'r') as hdf:
                    if 'fpico_mask' in hdf:
                        fpico_mask = hdf['fpico_mask'][:]
                    else:
                        fpico_mask = False

                print('output hdf file:', output_hdf_fpath)

                if isinstance(fpico_mask, np.ndarray):
                    with h5py.File(output_hdf_fpath, 'r+') as hdf:
                          hdf.create_dataset(name = 'fpico_mask', data = fpico_mask)
            else:
                pass
    else:
        pass

Perform deconvolutions on both detectors (where present).

In [5]:
for hdf_file in hdf_list:
    print('\n', hdf_file)
    xrf_deconvolute(hdf_file, 'falconx_det0', refit_state=True)
    xrf_deconvolute(hdf_file, 'fluodet_det0', refit_state=True)

print('\n All files processed')



 001_002_stitch
falconx_det0 node exists:  True
40200
small dataset z 201 y 200
spectra shape (40200, 4096)
reshaped spectra shape (201, 200, 4096)
spectra config file: 21_04_22_SDD_AXO_ls2893_1_002_stitch.cfg
output hdf file: C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs\per_detector_deconvolutions\001_002_stitch\falconx_det0\001_002_stitch.h5
fluodet_det0 node exists:  True
40200
small dataset z 201 y 200
spectra shape (40200, 4096)
reshaped spectra shape (201, 200, 4096)
spectra config file: 21_04_22_SDD_AXO_ls2893_1_002_stitch.cfg
output hdf file: C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs\per_detector_deconvolutions\001_002_stitch\fluodet_det0\001_002_stitch.h5

 001_004_stitch
falconx_det0 node exists:  True
40000
small dataset z 200 y 200
spectra shape (40000, 4096)
reshaped spectra shape (200, 200, 4096)
spectra config file: 21_04_22_SDD_AXO_ls2893_1_004_stitch.cfg
output hdf fil

## Concatenate chunked deconvolutions

In [6]:
# ID chunked deconvolutions
chunk_subdirs = list(out_dir.glob('*/*/*/chunk*'))
chunk_dirs = list(np.unique([i.parent for i in chunk_subdirs]))

# Find first hdf and copy to expected location
for chunk_dir in chunk_dirs:
    print(chunk_dir.parent.stem)

    # Identify chunks and concatenate
    hdf_chunks = list(chunk_dir.glob('*/*.h5'))    
    for i, hdf_chunk in enumerate(hdf_chunks[:]):
        if i == 0:
            hdf_base_path = chunk_dir / hdf_chunks[0].name
            shutil.copyfile(hdf_chunks[0], hdf_base_path) 
        else:
            print(hdf_chunk.parent.stem)
            with h5py.File(hdf_base_path, 'a') as hdf_base:
                with h5py.File(hdf_chunk, 'r') as hdf_add:
                    node = f"{hdf_base_path.stem}/plotselect"
                    for plot in hdf_base[node].keys():
                        subnode = f"{node}/{plot}"
                        array_top = hdf_base[subnode][:]
                        array_base = hdf_add[subnode][:]
                        array_stacked = np.concatenate([array_top, array_base], axis=0)
                        
                        del hdf_base[subnode]
                        hdf_base.create_dataset(subnode, data=array_stacked)
                        
    # Extract tifs from hdf channels
    with h5py.File(hdf_base_path, 'r+') as hdf:
        image_dir = hdf_base_path.parent / "IMAGES"
        image_dir.mkdir(parents=True, exist_ok=True)
        
        for img in hdf[node].keys():
            subnode = f"{node}/{img}"
            dset = hdf[subnode][:]
            outpath = str(image_dir / f'{img}.tiff')
            cv2.imwrite(outpath, dset)


breast_cancer_b2_solid_overview_0corrected_0003
chunk_1
chunk_2
chunk_3
chunk_4
chunk_5
chunk_6
chunk_7
chunk_8
chunk_9


 Incorporate step sizes for final channel image hdf files. 

In [7]:
# Incorporate key csv attributes per scan to deconvoluted hdfs for subsequent analysis
# hdf_img_fpaths = [i for i in out_dir.glob('*/*/*/*.h5')]
hdf_img_fpaths = [f"{hdf_file}.h5" for hdf_file in df_hdf_config['hdf_file']]
list(out_dir.glob(f"*/*/*/{hdf_img_fpaths[0]}"))[0]
hdf_img_fpaths = [list(out_dir.glob(f"*/*/*/{hdf_file}"))[0] for hdf_file in hdf_img_fpaths]

for hdf_img_fpath in hdf_img_fpaths:
    print(hdf_img_fpath)
    step = df_hdf_config.loc[df_hdf_config['hdf_file'] == hdf_img_fpath.stem, 'step_um'].iloc[0]
    det_type = df_hdf_config.loc[df_hdf_config['hdf_file'] == hdf_img_fpath.stem, 'detector'].iloc[0]
    dual_det = df_hdf_config.loc[df_hdf_config['hdf_file'] == hdf_img_fpath.stem, 'dual_detector'].iloc[0]
    
    if dual_det == 1:
        dual_det = 'yes'
    else:
        dual_det = 'no'
   
    with h5py.File(hdf_img_fpath, 'a') as hdf:
        if 'pixel_um' in hdf:
            pass 
        else:
            hdf.create_dataset(name = 'pixel_um', data = step)
        if 'detector' in hdf:
            print('detector in hdf, overwriting')
            del hdf['detector']
            hdf.create_dataset('detector', data=det_type)
        else:
            print('detector not in hdf, creating')
            hdf.create_dataset(name = 'detector', data = det_type)
        if 'dual_detector' in hdf:
            pass 
        else:
            hdf.create_dataset(name = 'dual_detector', data = dual_det)

print('Key acquisition metadata incorporated to .hdf files')

C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs\per_detector_deconvolutions\001_002_stitch\falconx_det0\001_002_stitch.h5
detector not in hdf, creating
C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs\per_detector_deconvolutions\001_004_stitch\falconx_det0\001_004_stitch.h5
detector not in hdf, creating
C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs\per_detector_deconvolutions\304_007_stitch\falconx_det0\304_007_stitch.h5
detector not in hdf, creating
C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs\per_detector_deconvolutions\304_009_stitch\falconx_det0\304_009_stitch.h5
detector not in hdf, creating
C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs\per_detector_deconvolutions\sample191_0001\falconx_det0\sample191_0001.h5
detector not in hdf, creating
C:\Users\M

Collate hdfs to summary hdfs per sample. For scans with dual detectors, element plots will be aggregated, while for single detector scans these will simply be copied over. 

In [8]:
detector_paths = ['falconx_det0', 'fluodet_det0']
out_sum_dir = out_dir / 'summary_hdfs'
out_sum_dir.mkdir(parents=True, exist_ok=True)

# Copy the falconX detector to summary scan output
for hdf_file in df_hdf_config['hdf_file']:
    hdf_dir = out_dir / 'per_detector_deconvolutions' / hdf_file 
    hdf_fpath = list(hdf_dir.glob('*/*.h5'))
    
    shutil.copyfile(hdf_fpath[0], out_sum_dir / hdf_fpath[0].name)
            
# For dual detector scans, modify the summary scan datasets by aggregating second detector data for plots
for hdf_file in df_hdf_config.loc[df_hdf_config['dual_detector'] == 1, 'hdf_file']:
    hdf_dir = out_dir / 'per_detector_deconvolutions' / hdf_file 
    hdf_fpath = list(hdf_dir.glob('*/*.h5'))
    
    hdf_base = h5py.File(hdf_fpath[0], 'r+')
    plots = list(hdf_base[f'{hdf_file}/plotselect'].keys())
    
    for plot in plots:
        dset = hdf_base[f'{hdf_file}/plotselect/{plot}'][:]
        
        with h5py.File(hdf_fpath[1], 'r+') as hdf_add:
            dset_add = hdf_add[f'{hdf_file}/plotselect/{plot}'][:]
            
        dset_mean = np.mean([dset, dset_add], axis=0)
        
        hdf_mod_path = out_sum_dir / hdf_fpath[0].name
        #print(hdf_mod_path.exists())
        with h5py.File(hdf_mod_path, 'r+') as hdf_mod:
            hdf_mod[f'{hdf_file}/plotselect/{plot}'][...] = dset_mean
        
    #print(dset_mean.shape) 
    hdf_base.close()
    
print('Summary hdfs output to: ', out_sum_dir)
    

Summary hdfs output to:  C:\Users\MerrickS\OneDrive\Work\2_UZH\Papers\1_MEZ_XRF\data\processed\xrf\2_deconvoluted_hdfs\summary_hdfs
